# Loading data with DisjointTimeBasedCesnetDataset

### Import

In [1]:
from tqdm import tqdm
import logging

from cesnet_tszoo.utils.enums import AgreggationType, SourceType, TimeFormat, DatasetType
from cesnet_tszoo.datasets import CESNET_TimeSeries24
from cesnet_tszoo.configs import DisjointTimeBasedConfig # Disjoint dataset MUST use DisjointTimeBasedConfig

### Setting logger

In [2]:
logging.basicConfig(
    level=logging.INFO,
    format="[%(asctime)s][%(name)s][%(levelname)s] - %(message)s")

### Preparing dataset

In [3]:
disjoint_dataset = CESNET_TimeSeries24.get_dataset(data_root="/some_directory/", source_type=SourceType.INSTITUTION_SUBNETS, aggregation=AgreggationType.AGG_1_HOUR, dataset_type=DatasetType.DISJOINT_TIME_BASED, display_details=True)

[2025-08-26 20:05:38,500][wrapper_dataset][INFO] - Dataset is disjoint_time_based. Use cesnet_tszoo.configs.DisjointTimeBasedConfig



Dataset details:

    AgreggationType.AGG_1_HOUR
        Time indices: range(0, 6717)
        Datetime: (datetime.datetime(2023, 10, 9, 0, 0, tzinfo=datetime.timezone.utc), datetime.datetime(2024, 7, 14, 21, 0, tzinfo=datetime.timezone.utc))

    SourceType.INSTITUTION_SUBNETS
        Time series indices: [0 1 2 3 4 ... 543 544 545 546 547], Length=548; use 'get_available_ts_indices' for full list
        Features with default values: {'n_flows': 0, 'n_packets': 0, 'n_bytes': 0, 'tcp_udp_ratio_packets': 0.5, 'tcp_udp_ratio_bytes': 0.5, 'dir_ratio_packets': 0.5, 'dir_ratio_bytes': 0.5, 'avg_duration': 0, 'avg_ttl': 0, 'sum_n_dest_asn': 0, 'avg_n_dest_asn': 0, 'std_n_dest_asn': 0, 'sum_n_dest_ports': 0, 'avg_n_dest_ports': 0, 'std_n_dest_ports': 0, 'sum_n_dest_ip': 0, 'avg_n_dest_ip': 0, 'std_n_dest_ip': 0}
        
        Additional data: ['ids_relationship', 'weekends_and_holidays']
        


### Loading data with DataLoader

- Load data using Pytorch Dataloader.
- Last batch is never dropped (unless sliding_window is used).
- Workers affect how many processes will be used for loading data for specific set.
    - Workers set to 0, means loading will be ran on main process.
    - Set workers can be overriden in `get_*_dataloader` with parameter `workers`.
- Batch size affect how many times for every time series will be in one batch (differs when sliding window is used).
- Batch consists of: (only when sliding window is not used).
    - When `time_format` is not TimeFormat.DATETIME, then batch is one Numpy array of shape `(train_ts/val_ts/test_ts, batch_size, features_to_take + used ids)`.
    - When `time_format` is TimeFormat.DATETIME, then batch is a tuple: (Numpy array of shape `(train_ts/val_ts/test_ts, batch_size, features_to_take + used ids (without time))`, Numpy array of shape `(batch_size)`)

In [4]:
config = DisjointTimeBasedConfig(train_ts=0.5, val_ts=0.2, test_ts=0.1, train_time_period=0.5, val_time_period=0.3, test_time_period=0.2, features_to_take="all", time_format=TimeFormat.ID_TIME,
                         train_workers=0, val_workers=0, test_workers=0, init_workers=0,
                         train_batch_size=32, val_batch_size=64, test_batch_size=128)
disjoint_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-08-26 20:05:38,506][disjoint_time_based_config][INFO] - Quick validation succeeded.
[2025-08-26 20:05:38,526][disjoint_time_based_config][INFO] - Finalization and validation completed successfully.
[2025-08-26 20:05:38,531][cesnet_dataset][INFO] - Updating config for train set.
100%|██████████| 274/274 [00:00<00:00, 1203.13it/s]
[2025-08-26 20:05:38,777][cesnet_dataset][INFO] - Updating config for val set.
100%|██████████| 109/109 [00:00<00:00, 1128.30it/s]
[2025-08-26 20:05:38,886][cesnet_dataset][INFO] - Updating config for test set.
100%|██████████| 54/54 [00:00<00:00, 1087.35it/s]
[2025-08-26 20:05:38,942][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_1_HOUR
    Source: SourceType.INSTITUTION_SUBNETS

    Time series
        Train time series IDs: [115 312 383 212 183 ... 284 194 353   4 307], Length=274
        Val time series IDs: [172  81 226 109 267 ... 165 277 465 314  11], Length=109
        Test time series IDs: [321 460 155 295 379 ... 429 178 189  28 144], Length=54
    Time periods
        Train time periods: range(0, 3359)
        Val time periods: range(3359, 5374)
        Test time periods: range(5374, 6717)
    Features
        Taken features: ['n_flows', 'n_packets', 'n_bytes', 'sum_n_dest_asn', 'avg_n_dest_asn', 'std_n_dest_asn', 'sum_n_dest_ports', 'avg_n_dest_ports', 'std_n_dest_ports', 'sum_n_dest_ip', 'avg_n_dest_ip', 'std_n_dest_ip', 'tcp_udp_ratio_packets', 'tcp_udp_ratio_bytes', 'dir_ratio_packets', 'dir_ratio_bytes', 'avg_duration', 'avg_ttl']
        Default values: [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 

You can also change set batch sizes later with `update_dataset_config_and_initialize` or `set_batch_sizes`.

In [5]:
disjoint_dataset.update_dataset_config_and_initialize(train_batch_size=33, val_batch_size=65, test_batch_size="config")
# Or
disjoint_dataset.set_batch_sizes(train_batch_size=33, val_batch_size=65, test_batch_size="config")

[2025-08-26 20:05:38,948][cesnet_dataset][INFO] - Re-initialization is not needed.
[2025-08-26 20:05:38,949][cesnet_dataset][INFO] - Configuration has been changed successfuly.
[2025-08-26 20:05:38,950][cesnet_dataset][INFO] - Re-initialization is not needed.
[2025-08-26 20:05:38,950][cesnet_dataset][INFO] - Configuration has been changed successfuly.
[2025-08-26 20:05:38,951][cesnet_dataset][INFO] - Batch sizes has been changed successfuly.


You can also change set workers later with `update_dataset_config_and_initialize` or `set_workers`.

In [6]:
disjoint_dataset.update_dataset_config_and_initialize(train_workers=0, val_workers=0, test_workers=0, init_workers=0)
# Or
disjoint_dataset.set_workers(train_workers=0, val_workers=0, test_workers=0, init_workers=0)

[2025-08-26 20:05:38,956][cesnet_dataset][INFO] - Re-initialization is not needed.
[2025-08-26 20:05:38,956][cesnet_dataset][INFO] - Configuration has been changed successfuly.
[2025-08-26 20:05:38,957][cesnet_dataset][INFO] - Re-initialization is not needed.
[2025-08-26 20:05:38,958][cesnet_dataset][INFO] - Configuration has been changed successfuly.
[2025-08-26 20:05:38,958][cesnet_dataset][INFO] - Workers has been changed successfuly.


#### Train set

- Affected by `train_batch_size`.
- Affected by `train_workers`.

In [7]:
dataloader = disjoint_dataset.get_train_dataloader(workers="config")

batches = []

for batch in tqdm(dataloader):
    batches.append(batch)
    
display(batches[0].shape)

[2025-08-26 20:05:38,967][cesnet_dataset][INFO] - Created new cached train_dataloader.
100%|██████████| 105/105 [00:02<00:00, 44.43it/s]


(274, 32, 20)

#### Val set

- Affected by `val_batch_size`.
- Affected by `val_workers`.

In [8]:
dataloader = disjoint_dataset.get_val_dataloader(workers="config")

batches = []

for batch in tqdm(dataloader):
    batches.append(batch)
    
display(batches[0].shape)

[2025-08-26 20:05:41,341][cesnet_dataset][INFO] - Created new cached val_dataloader.
100%|██████████| 32/32 [00:00<00:00, 66.32it/s]


(109, 64, 20)

#### Test set

- Affected by `test_batch_size`.
- Affected by `test_workers`.

In [9]:
dataloader = disjoint_dataset.get_test_dataloader(workers="config")

batches = []

for batch in tqdm(dataloader):
    batches.append(batch)
    
display(batches[0].shape)

[2025-08-26 20:05:41,840][cesnet_dataset][INFO] - Created new cached test_dataloader.
100%|██████████| 11/11 [00:00<00:00, 95.58it/s]


(54, 128, 20)

#### Using time_format=TimeFormat.DATETIME

In [10]:
config = DisjointTimeBasedConfig(train_ts=0.5, val_ts=0.2, test_ts=0.1, train_time_period=0.5, val_time_period=0.3, test_time_period=0.2, features_to_take="all", time_format=TimeFormat.DATETIME,
                         train_workers=0, val_workers=0, test_workers=0, init_workers=0,
                         train_batch_size=32, val_batch_size=64, test_batch_size=128)
disjoint_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-08-26 20:05:41,965][disjoint_time_based_config][INFO] - Quick validation succeeded.
[2025-08-26 20:05:41,988][disjoint_time_based_config][INFO] - Finalization and validation completed successfully.
[2025-08-26 20:05:41,991][cesnet_dataset][INFO] - Updating config for train set.
100%|██████████| 274/274 [00:00<00:00, 2612.29it/s]
[2025-08-26 20:05:42,102][cesnet_dataset][INFO] - Updating config for val set.
100%|██████████| 109/109 [00:00<00:00, 3357.22it/s]
[2025-08-26 20:05:42,140][cesnet_dataset][INFO] - Updating config for test set.
100%|██████████| 54/54 [00:00<00:00, 2697.40it/s]
[2025-08-26 20:05:42,161][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_1_HOUR
    Source: SourceType.INSTITUTION_SUBNETS

    Time series
        Train time series IDs: [329 105   6 535 131 ... 274 441 116 356 499], Length=274
        Val time series IDs: [247 309 459 129 229 ... 540 287 352 418 167], Length=109
        Test time series IDs: [256 306 237 337  37 ...  11 244  22  67 446], Length=54
    Time periods
        Train time periods: range(0, 3359)
        Val time periods: range(3359, 5374)
        Test time periods: range(5374, 6717)
    Features
        Taken features: ['n_flows', 'n_packets', 'n_bytes', 'sum_n_dest_asn', 'avg_n_dest_asn', 'std_n_dest_asn', 'sum_n_dest_ports', 'avg_n_dest_ports', 'std_n_dest_ports', 'sum_n_dest_ip', 'avg_n_dest_ip', 'std_n_dest_ip', 'tcp_udp_ratio_packets', 'tcp_udp_ratio_bytes', 'dir_ratio_packets', 'dir_ratio_bytes', 'avg_duration', 'avg_ttl']
        Default values: [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 

In [11]:
dataloader = disjoint_dataset.get_train_dataloader(workers="config")

batches = []

for batch in tqdm(dataloader):
    batches.append(batch)
    
display(batches[0][0].shape) # data without time
display(batches[0][1].shape) # time

[2025-08-26 20:05:42,170][cesnet_dataset][INFO] - Created new cached train_dataloader.
100%|██████████| 105/105 [00:02<00:00, 51.07it/s]


(274, 32, 19)

(32,)

#### Specifying which time series to load

- Every `get_*_dataloader` has parameter `ts_id`.
    - When `ts_id` is None, then it returns as previous examples.
    - When `ts_id` is not None, then it returns only one time series of that specified id.

In [12]:
config = DisjointTimeBasedConfig(train_ts=[177, 176, 319, 267], val_ts=None, test_ts=None, train_time_period=0.5, features_to_take="all", time_format=TimeFormat.ID_TIME,
                         train_workers=0, val_workers=0, test_workers=0, init_workers=0,
                         train_batch_size=32, val_batch_size=64, test_batch_size=128)
disjoint_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-08-26 20:05:44,235][disjoint_time_based_config][INFO] - Quick validation succeeded.
[2025-08-26 20:05:44,245][disjoint_time_based_config][INFO] - Finalization and validation completed successfully.
[2025-08-26 20:05:44,248][cesnet_dataset][INFO] - Updating config for train set.
100%|██████████| 4/4 [00:00<00:00, 2001.34it/s]
[2025-08-26 20:05:44,252][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_1_HOUR
    Source: SourceType.INSTITUTION_SUBNETS

    Time series
        Train time series IDs: [177 176 319 267], Length=4
        Val time series IDs: None
        Test time series IDs: None
    Time periods
        Train time periods: range(0, 3359)
        Val time periods: None
        Test time periods: None
    Features
        Taken features: ['n_flows', 'n_packets', 'n_bytes', 'sum_n_dest_asn', 'avg_n_dest_asn', 'std_n_dest_asn', 'sum_n_dest_ports', 'avg_n_dest_ports', 'std_n_dest_ports', 'sum_n_dest_ip', 'avg_n_dest_ip', 'std_n_dest_ip', 'tcp_udp_ratio_packets', 'tcp_udp_ratio_bytes', 'dir_ratio_packets', 'dir_ratio_bytes', 'avg_duration', 'avg_ttl']
        Default values: [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.5 0.5 0.5 0.5 0.  0. ]
        Time series ID included: True
        Time included: True    
        Time format: TimeFormat.ID_TIME
    Sliding window
      

In [13]:
dataloader = disjoint_dataset.get_train_dataloader(ts_id=177 , workers="config",)

batches = []

for batch in tqdm(dataloader):
    batches.append(batch)
    
display(batches[0].shape)

[2025-08-26 20:05:44,261][cesnet_dataset][INFO] - Created new cached train_dataloader.
100%|██████████| 105/105 [00:00<00:00, 2136.05it/s]


(1, 32, 20)

#### Sliding window

- Both `sliding_window_size` and `sliding_window_prediction_size` must be set if you want to use sliding window.
- Batch sizes are used for background caching.
- Batch consists of:
    - When `time_format` is not TimeFormat.DATETIME, then batch is a tuple: <br>
    ( <br>
        Numpy array of shape `(train_ts/val_ts/test_ts, sliding_window_size, features_to_take + used ids)`, <br>
        Numpy array of shape `(train_ts/val_ts/test_ts, sliding_window_prediction_size, features_to_take + used ids)` <br>
    ).
    - When `time_format` is TimeFormat.DATETIME, then batch is a tuple: <br>
    ( <br>
        Numpy array of shape `(train_ts/val_ts/test_ts, sliding_window_size, features_to_take + used ids (without time))`, <br>
        Numpy array of shape `(train_ts/val_ts/test_ts, sliding_window_prediction_size, features_to_take + used ids (without time))`, <br>
        Numpy array of shape `(sliding_window_size)` of time, <br>
        Numpy array of shape `(sliding_window_prediction_size)` of time, <br>
    ).
- You can modify sliding window step size with `sliding_window_step`
- You can use `set_shared_size` to set how many times time periods should share.
    - `val_time_period` takes from `train_time_period`
    - `test_time_period` takes from `val_time_period` or `train_time_period`

In [14]:
config = DisjointTimeBasedConfig(train_ts=0.5, val_ts=0.2, test_ts=0.1, train_time_period=range(0, 1000), val_time_period=range(1000, 1500), test_time_period=range(1500, 2000), features_to_take=["n_flows"], time_format=TimeFormat.ID_TIME,
                         train_workers=0, val_workers=0, test_workers=0, init_workers=0,
                         train_batch_size=32, val_batch_size=64, test_batch_size=128,
                         sliding_window_size=22, sliding_window_prediction_size=2, sliding_window_step=2, set_shared_size=0.05)
disjoint_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-08-26 20:05:44,327][disjoint_time_based_config][INFO] - Quick validation succeeded.
[2025-08-26 20:05:44,347][disjoint_time_based_config][INFO] - Finalization and validation completed successfully.
[2025-08-26 20:05:44,351][cesnet_dataset][INFO] - Updating config for train set.
100%|██████████| 274/274 [00:00<00:00, 6531.62it/s]
[2025-08-26 20:05:44,399][cesnet_dataset][INFO] - Updating config for val set.
100%|██████████| 109/109 [00:00<00:00, 5001.63it/s]
[2025-08-26 20:05:44,426][cesnet_dataset][INFO] - Updating config for test set.
100%|██████████| 54/54 [00:00<00:00, 5677.64it/s]
[2025-08-26 20:05:44,436][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_1_HOUR
    Source: SourceType.INSTITUTION_SUBNETS

    Time series
        Train time series IDs: [156   9 318  59 233 ...   4  66  83 417 416], Length=274
        Val time series IDs: [185 354  44 381 266 ... 107 406 138 447 216], Length=109
        Test time series IDs: [391  24 353  82 498 ... 226 254 504 179 357], Length=54
    Time periods
        Train time periods: range(0, 1000)
        Val time periods: range(665, 1500)
        Test time periods: range(1165, 2000)
    Features
        Taken features: ['n_flows']
        Default values: [0.]
        Time series ID included: True
        Time included: True    
        Time format: TimeFormat.ID_TIME
    Sliding window
        Sliding window size: 22
        Sliding window prediction size: 2
        Sliding window step size: 2
    Fillers
        Filler type: None
    Transformers
        Transformer type: None
    Anomaly handler
   

In [15]:
dataloader = disjoint_dataset.get_train_dataloader(workers="config")

batches = []

for sliding_window, sliding_window_prediction in tqdm(dataloader):
    batches.append((sliding_window, sliding_window_prediction))

[2025-08-26 20:05:44,445][cesnet_dataset][INFO] - Created new cached train_dataloader.
100%|██████████| 489/489 [00:00<00:00, 1125.03it/s]


You can also change sliding window parameters later with `update_dataset_config_and_initialize` or `set_sliding_window`.

In [16]:
disjoint_dataset.update_dataset_config_and_initialize(sliding_window_size=22, sliding_window_prediction_size=3, sliding_window_step="config", set_shared_size="config", workers=0)
# Or
disjoint_dataset.set_sliding_window(sliding_window_size=22, sliding_window_prediction_size=3, sliding_window_step="config", set_shared_size="config", workers=0)

[2025-08-26 20:05:44,886][cesnet_dataset][INFO] - Re-initialization is not needed.
[2025-08-26 20:05:44,886][disjoint_time_based_config][INFO] - all_batch_size adjusted to 25 as it should be greater than or equal to sliding_window_size + sliding_window_prediction_size.
[2025-08-26 20:05:44,887][cesnet_dataset][INFO] - Destroyed cached train_dataloader.
[2025-08-26 20:05:44,887][cesnet_dataset][INFO] - Configuration has been changed successfuly.
[2025-08-26 20:05:44,888][cesnet_dataset][INFO] - Re-initialization is not needed.
[2025-08-26 20:05:44,888][cesnet_dataset][INFO] - Configuration has been changed successfuly.
[2025-08-26 20:05:44,889][cesnet_dataset][INFO] - Sliding window values has been changed successfuly.


##### Using time_format=TimeFormat.DATETIME

In [17]:
config = DisjointTimeBasedConfig(train_ts=0.5, val_ts=0.2, test_ts=0.1, train_time_period=range(0, 1000), val_time_period=range(978, 1500), test_time_period=range(1478, 2000), features_to_take=["n_flows"], time_format=TimeFormat.DATETIME,
                         train_workers=0, val_workers=0, test_workers=0, init_workers=0,
                         train_batch_size=32, val_batch_size=64, test_batch_size=128,
                         sliding_window_size=22, sliding_window_prediction_size=2, sliding_window_step=2, set_shared_size=100)
disjoint_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-08-26 20:05:44,893][disjoint_time_based_config][INFO] - Quick validation succeeded.
[2025-08-26 20:05:44,913][disjoint_time_based_config][INFO] - Finalization and validation completed successfully.
[2025-08-26 20:05:44,916][cesnet_dataset][INFO] - Updating config for train set.
100%|██████████| 274/274 [00:00<00:00, 8245.90it/s]
[2025-08-26 20:05:44,956][cesnet_dataset][INFO] - Updating config for val set.
100%|██████████| 109/109 [00:00<00:00, 7506.06it/s]
[2025-08-26 20:05:44,976][cesnet_dataset][INFO] - Updating config for test set.
100%|██████████| 54/54 [00:00<00:00, 7192.52it/s]
[2025-08-26 20:05:44,985][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_1_HOUR
    Source: SourceType.INSTITUTION_SUBNETS

    Time series
        Train time series IDs: [103 530 396  39 356 ... 461 265  98 360 166], Length=274
        Val time series IDs: [271 122 138 301 407 ... 527  60 226 220 242], Length=109
        Test time series IDs: [405 284 110 539 241 ... 222 225 503  86 487], Length=54
    Time periods
        Train time periods: range(0, 1000)
        Val time periods: range(900, 1500)
        Test time periods: range(1400, 2000)
    Features
        Taken features: ['n_flows']
        Default values: [0.]
        Time series ID included: True
        Time included: True    
        Time format: TimeFormat.DATETIME
    Sliding window
        Sliding window size: 22
        Sliding window prediction size: 2
        Sliding window step size: 2
    Fillers
        Filler type: None
    Transformers
        Transformer type: None
    Anomaly handler
  

In [18]:
dataloader = disjoint_dataset.get_train_dataloader(workers="config")

batches = []

for sliding_window, sliding_window_prediction, sliding_window_times, sliding_window_prediction_times in tqdm(dataloader):
    batches.append((sliding_window, sliding_window_prediction, sliding_window_times, sliding_window_prediction_times))

[2025-08-26 20:05:44,996][cesnet_dataset][INFO] - Created new cached train_dataloader.
100%|██████████| 489/489 [00:00<00:00, 1097.11it/s]


### Loading data as Dataframe

- Batch size has no effect.
- Sliding window has no effect.
- Returns every time series in `train_ts/val_ts/test_ts` with sets specified time period.
- Data is returned as Pandas Dataframe.
- Workers affect how many processes will be used for loading data for specific set.
    - Workers set to 0, means loading will be ran on main process.
    - Set workers can be overriden in `get_*_df` with parameter `workers`.

In [19]:
config = DisjointTimeBasedConfig(train_ts=0.5, val_ts=0.2, test_ts=0.1, train_time_period=0.5, val_time_period=0.3, test_time_period=0.2, features_to_take="all", time_format=TimeFormat.ID_TIME,
                         train_workers=0, val_workers=0, test_workers=0, init_workers=0)
disjoint_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-08-26 20:05:45,446][disjoint_time_based_config][INFO] - Quick validation succeeded.
[2025-08-26 20:05:45,466][disjoint_time_based_config][INFO] - Finalization and validation completed successfully.
[2025-08-26 20:05:45,470][cesnet_dataset][INFO] - Updating config for train set.
100%|██████████| 274/274 [00:00<00:00, 2683.70it/s]
[2025-08-26 20:05:45,576][cesnet_dataset][INFO] - Updating config for val set.
100%|██████████| 109/109 [00:00<00:00, 3278.87it/s]
[2025-08-26 20:05:45,614][cesnet_dataset][INFO] - Updating config for test set.
100%|██████████| 54/54 [00:00<00:00, 3647.93it/s]
[2025-08-26 20:05:45,631][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_1_HOUR
    Source: SourceType.INSTITUTION_SUBNETS

    Time series
        Train time series IDs: [ 82 213  56 365 195 ...  26 262 170 325 394], Length=274
        Val time series IDs: [371 471 149 244  75 ... 479   1 261  92  34], Length=109
        Test time series IDs: [472 512 522 287 432 ... 206 426 276 182 421], Length=54
    Time periods
        Train time periods: range(0, 3359)
        Val time periods: range(3359, 5374)
        Test time periods: range(5374, 6717)
    Features
        Taken features: ['n_flows', 'n_packets', 'n_bytes', 'sum_n_dest_asn', 'avg_n_dest_asn', 'std_n_dest_asn', 'sum_n_dest_ports', 'avg_n_dest_ports', 'std_n_dest_ports', 'sum_n_dest_ip', 'avg_n_dest_ip', 'std_n_dest_ip', 'tcp_udp_ratio_packets', 'tcp_udp_ratio_bytes', 'dir_ratio_packets', 'dir_ratio_bytes', 'avg_duration', 'avg_ttl']
        Default values: [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 

#### Train set

- Affected by `train_workers`.

In [20]:
df = disjoint_dataset.get_train_df(as_single_dataframe=True, workers="config")
dfs = disjoint_dataset.get_train_df(as_single_dataframe=False, workers="config")

df.head(10)

,id_institution_subnet,id_time,n_flows,n_packets,n_bytes,sum_n_dest_asn,avg_n_dest_asn,std_n_dest_asn,sum_n_dest_ports,avg_n_dest_ports,std_n_dest_ports,sum_n_dest_ip,avg_n_dest_ip,std_n_dest_ip,tcp_udp_ratio_packets,tcp_udp_ratio_bytes,dir_ratio_packets,dir_ratio_bytes,avg_duration,avg_ttl
0,82.0,0.0,675.0,873.0,35960.0,501.0,4.96,2.09,515.0,5.10,2.23,642.0,6.36,3.32,0.740234,0.729980,0.409912,0.399902,1.60,192.190002
1,82.0,1.0,800.0,994.0,42194.0,545.0,5.34,1.66,570.0,5.59,1.88,760.0,7.45,3.54,0.669922,0.640137,0.379883,0.370117,1.61,198.740005
2,82.0,2.0,679.0,865.0,36663.0,471.0,4.71,1.78,482.0,4.82,2.03,658.0,6.58,3.40,0.680176,0.669922,0.409912,0.399902,1.66,194.520004
3,82.0,3.0,614.0,806.0,34400.0,433.0,4.08,1.78,441.0,4.16,1.82,579.0,5.46,3.18,0.660156,0.660156,0.479980,0.479980,1.57,187.979996
4,82.0,4.0,413.0,553.0,22524.0,312.0,3.47,1.69,308.0,3.42,1.63,400.0,4.44,2.29,0.629883,0.629883,0.449951,0.449951,2.51,191.220001
5,82.0,5.0,421.0,556.0,25592.0,296.0,3.36,1.58,293.0,3.33,1.54,407.0,4.62,3.18,0.669922,0.660156,0.479980,0.469971,1.67,189.789993
6,82.0,6.0,327.0,465.0,22889.0,262.0,3.12,1.51,261.0,3.11,1.54,320.0,3.81,2.29,0.689941,0.660156,0.529785,0.509766,2.34,186.479996
7,82.0,7.0,461.0,607.0,24567.0,338.0,3.67,1.42,336.0,3.65,1.36,453.0,4.92,2.47,0.680176,0.680176,0.459961,0.459961,2.07,192.330002
8,82.0,8.0,443.0,573.0,25998.0,322.0,3.66,1.58,320.0,3.64,1.63,427.0,4.85,2.84,0.660156,0.640137,0.469971,0.449951,1.33,172.460007
9,82.0,9.0,340.0,457.0,20739.0,260.0,3.10,1.67,245.0,2.92,1.63,328.0,3.90,2.44,0.609863,0.589844,0.520020,0.500000,2.50,186.490005


In [21]:
dfs

[      id_institution_subnet  id_time  n_flows  n_packets  n_bytes  \
 0                      82.0      0.0    675.0      873.0  35960.0   
 1                      82.0      1.0    800.0      994.0  42194.0   
 2                      82.0      2.0    679.0      865.0  36663.0   
 3                      82.0      3.0    614.0      806.0  34400.0   
 4                      82.0      4.0    413.0      553.0  22524.0   
 ...                     ...      ...      ...        ...      ...   
 3354                   82.0   3354.0    479.0      701.0  30551.0   
 3355                   82.0   3355.0    482.0      732.0  32803.0   
 3356                   82.0   3356.0    577.0      872.0  37154.0   
 3357                   82.0   3357.0    571.0      869.0  42498.0   
 3358                   82.0   3358.0    470.0      734.0  36412.0   
 
       sum_n_dest_asn  avg_n_dest_asn  std_n_dest_asn  sum_n_dest_ports  \
 0              501.0            4.96            2.09             515.0   
 1      

#### Val set

- Affected by `val_workers`.

In [22]:
df = disjoint_dataset.get_val_df(as_single_dataframe=True, workers="config")
dfs = disjoint_dataset.get_val_df(as_single_dataframe=False, workers="config")

df.head(10)

,id_institution_subnet,id_time,n_flows,n_packets,n_bytes,sum_n_dest_asn,avg_n_dest_asn,std_n_dest_asn,sum_n_dest_ports,avg_n_dest_ports,std_n_dest_ports,sum_n_dest_ip,avg_n_dest_ip,std_n_dest_ip,tcp_udp_ratio_packets,tcp_udp_ratio_bytes,dir_ratio_packets,dir_ratio_bytes,avg_duration,avg_ttl
0,371.0,3359.0,917.0,15862.0,6663042.0,289.0,7.05,3.25,351.0,8.56,5.45,641.0,15.630000,15.490000,0.819824,0.830078,0.489990,0.569824,45.200001,126.129997
1,371.0,3360.0,970.0,30351.0,22225741.0,266.0,5.91,3.33,312.0,6.93,4.16,638.0,14.180000,16.879999,0.839844,0.859863,0.500000,0.560059,37.630001,121.589996
2,371.0,3361.0,1035.0,25953.0,17967459.0,274.0,5.59,3.63,337.0,6.88,4.80,694.0,14.160000,18.790001,0.830078,0.859863,0.509766,0.569824,30.900000,131.059998
3,371.0,3362.0,986.0,16080.0,6900281.0,278.0,6.62,3.62,327.0,7.79,5.09,665.0,15.830000,19.160000,0.850098,0.890137,0.500000,0.609863,28.760000,131.330002
4,371.0,3363.0,923.0,23287.0,15174408.0,264.0,6.00,4.10,293.0,6.66,4.56,622.0,14.140000,18.440001,0.819824,0.859863,0.529785,0.620117,37.150002,126.669998
5,371.0,3364.0,706.0,13141.0,5186896.0,246.0,5.59,3.90,280.0,6.36,4.75,509.0,11.570000,13.860000,0.839844,0.859863,0.549805,0.660156,35.389999,120.750000
6,371.0,3365.0,1023.0,70010.0,52160913.0,324.0,7.53,7.33,286.0,6.65,3.68,773.0,17.980000,30.760000,0.870117,0.870117,0.600098,0.649902,38.369999,129.820007
7,371.0,3366.0,1592.0,88362.0,79514204.0,312.0,7.43,8.52,278.0,6.62,4.79,1084.0,25.809999,49.959999,0.879883,0.910156,0.569824,0.620117,35.490002,116.900002
8,371.0,3367.0,2580.0,298586.0,297816710.0,441.0,10.76,11.94,364.0,8.88,5.22,1670.0,40.730000,80.559998,0.890137,0.899902,0.520020,0.580078,36.779999,120.580002
9,371.0,3368.0,2296.0,216745.0,194491117.0,440.0,9.17,12.49,346.0,7.21,4.94,1616.0,33.669998,73.269997,0.899902,0.910156,0.500000,0.569824,34.599998,129.529999


In [23]:
dfs

[      id_institution_subnet  id_time  n_flows  n_packets     n_bytes  \
 0                     371.0   3359.0    917.0    15862.0   6663042.0   
 1                     371.0   3360.0    970.0    30351.0  22225741.0   
 2                     371.0   3361.0   1035.0    25953.0  17967459.0   
 3                     371.0   3362.0    986.0    16080.0   6900281.0   
 4                     371.0   3363.0    923.0    23287.0  15174408.0   
 ...                     ...      ...      ...        ...         ...   
 2010                  371.0   5369.0   2870.0    82786.0  33924575.0   
 2011                  371.0   5370.0   2818.0    47724.0  24931911.0   
 2012                  371.0   5371.0   2819.0    47081.0  22246314.0   
 2013                  371.0   5372.0   3146.0    39479.0  17449050.0   
 2014                  371.0   5373.0   4141.0    61013.0  36996059.0   
 
       sum_n_dest_asn  avg_n_dest_asn  std_n_dest_asn  sum_n_dest_ports  \
 0              289.0            7.05          

#### Test set

- Affected by `test_workers`.

In [24]:
df = disjoint_dataset.get_test_df(as_single_dataframe=True, workers="config")
dfs = disjoint_dataset.get_test_df(as_single_dataframe=False, workers="config")

df.head(10)

,id_institution_subnet,id_time,n_flows,n_packets,n_bytes,sum_n_dest_asn,avg_n_dest_asn,std_n_dest_asn,sum_n_dest_ports,avg_n_dest_ports,std_n_dest_ports,sum_n_dest_ip,avg_n_dest_ip,std_n_dest_ip,tcp_udp_ratio_packets,tcp_udp_ratio_bytes,dir_ratio_packets,dir_ratio_bytes,avg_duration,avg_ttl
0,472.0,5374.0,18.0,24.0,1014.0,16.0,2.67,1.37,17.0,2.83,1.17,17.0,2.83,1.17,0.899902,0.919922,0.429932,0.409912,0.00,171.179993
1,472.0,5375.0,9.0,10.0,420.0,9.0,2.25,0.96,9.0,2.25,0.96,9.0,2.25,0.96,0.750000,0.750000,0.790039,0.790039,0.00,223.500000
2,472.0,5376.0,12.0,19.0,1541.0,12.0,3.00,0.82,11.0,2.75,0.96,12.0,3.00,0.82,0.660156,0.560059,0.270020,0.170044,0.33,124.849998
3,472.0,5377.0,19.0,23.0,1000.0,14.0,2.80,1.10,19.0,3.80,1.79,18.0,3.60,1.52,0.970215,0.959961,0.379883,0.379883,0.01,215.050003
4,472.0,5378.0,9.0,20.0,1304.0,8.0,2.00,0.82,9.0,2.25,0.50,9.0,2.25,0.50,0.469971,0.429932,0.320068,0.340088,1.14,210.000000
5,472.0,5379.0,11.0,15.0,680.0,9.0,3.00,1.00,11.0,3.67,1.53,11.0,3.67,1.53,0.950195,0.939941,0.350098,0.330078,0.03,220.869995
6,472.0,5380.0,5.0,7.0,316.0,3.0,1.00,1.00,5.0,1.67,0.58,5.0,1.67,0.58,0.609863,0.580078,0.439941,0.439941,0.00,185.000000
7,472.0,5381.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.500000,0.500000,0.500000,0.500000,0.00,0.000000
8,472.0,5382.0,10.0,14.0,609.0,10.0,3.33,2.31,10.0,3.33,2.31,10.0,3.33,2.31,0.709961,0.689941,0.330078,0.350098,0.02,195.779999
9,472.0,5383.0,11.0,15.0,780.0,8.0,2.00,0.82,11.0,2.75,1.26,11.0,2.75,1.26,0.750000,0.729980,0.500000,0.529785,0.00,182.559998


In [25]:
dfs

[      id_institution_subnet  id_time  n_flows  n_packets  n_bytes  \
 0                     472.0   5374.0     18.0       24.0   1014.0   
 1                     472.0   5375.0      9.0       10.0    420.0   
 2                     472.0   5376.0     12.0       19.0   1541.0   
 3                     472.0   5377.0     19.0       23.0   1000.0   
 4                     472.0   5378.0      9.0       20.0   1304.0   
 ...                     ...      ...      ...        ...      ...   
 1338                  472.0   6712.0     13.0       16.0    757.0   
 1339                  472.0   6713.0     22.0       27.0   1176.0   
 1340                  472.0   6714.0     16.0       25.0   1060.0   
 1341                  472.0   6715.0     25.0       42.0   2268.0   
 1342                  472.0   6716.0     30.0       40.0   1764.0   
 
       sum_n_dest_asn  avg_n_dest_asn  std_n_dest_asn  sum_n_dest_ports  \
 0               16.0            2.67            1.37              17.0   
 1      

### Loading data as singular Numpy array 

- Batch size has no effect.
- Sliding window has no effect.
- Returns every time series in `train_ts/val_ts/test_ts` with sets specified time period.
- Data is returned as one Numpy array.
- Follows similar rules to Dataloader batches, regarding shape (excluding sliding window parameters).
- Workers affect how many processes will be used for loading data for specific set.
    - Workers set to 0, means loading will be ran on main process.
    - Set workers can be overriden in `get_*_numpy` with parameter `workers`.

In [26]:
config = DisjointTimeBasedConfig(train_ts=0.5, val_ts=0.2, test_ts=0.1, train_time_period=0.5, val_time_period=0.3, test_time_period=0.2, features_to_take="all", time_format=TimeFormat.ID_TIME,
                         train_workers=0, val_workers=0, test_workers=0, init_workers=0)
disjoint_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-08-26 20:05:48,173][disjoint_time_based_config][INFO] - Quick validation succeeded.
[2025-08-26 20:05:48,192][disjoint_time_based_config][INFO] - Finalization and validation completed successfully.
[2025-08-26 20:05:48,195][cesnet_dataset][INFO] - Updating config for train set.
100%|██████████| 274/274 [00:00<00:00, 2690.15it/s]
[2025-08-26 20:05:48,301][cesnet_dataset][INFO] - Updating config for val set.
100%|██████████| 109/109 [00:00<00:00, 3187.67it/s]
[2025-08-26 20:05:48,341][cesnet_dataset][INFO] - Updating config for test set.
100%|██████████| 54/54 [00:00<00:00, 3722.57it/s]
[2025-08-26 20:05:48,358][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_1_HOUR
    Source: SourceType.INSTITUTION_SUBNETS

    Time series
        Train time series IDs: [318  51  53 274 222 ... 413 466 142  47  46], Length=274
        Val time series IDs: [251 347 330 223 292 ... 543 178 339 264 493], Length=109
        Test time series IDs: [407 237 445  66 510 ... 463 135 422 406 217], Length=54
    Time periods
        Train time periods: range(0, 3359)
        Val time periods: range(3359, 5374)
        Test time periods: range(5374, 6717)
    Features
        Taken features: ['n_flows', 'n_packets', 'n_bytes', 'sum_n_dest_asn', 'avg_n_dest_asn', 'std_n_dest_asn', 'sum_n_dest_ports', 'avg_n_dest_ports', 'std_n_dest_ports', 'sum_n_dest_ip', 'avg_n_dest_ip', 'std_n_dest_ip', 'tcp_udp_ratio_packets', 'tcp_udp_ratio_bytes', 'dir_ratio_packets', 'dir_ratio_bytes', 'avg_duration', 'avg_ttl']
        Default values: [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 

#### Train set

- Affected by `train_workers`.

In [27]:
numpy_array = disjoint_dataset.get_train_numpy(workers="config")

display(numpy_array.shape)

(274, 3359, 20)

#### Val set

- Affected by `val_workers`.

In [28]:
numpy_array = disjoint_dataset.get_val_numpy(workers="config")

display(numpy_array.shape)

(109, 2015, 20)

#### Test set

- Affected by `test_workers`.

In [29]:
numpy_array = disjoint_dataset.get_test_numpy(workers="config")

display(numpy_array.shape)

(54, 1343, 20)

#### Using time_format=TimeFormat.DATETIME

In [30]:
config = DisjointTimeBasedConfig(train_ts=0.5, val_ts=0.2, test_ts=0.1, train_time_period=0.5, val_time_period=0.3, test_time_period=0.2, features_to_take="all", time_format=TimeFormat.DATETIME,
                         train_workers=0, val_workers=0, test_workers=0, init_workers=0)
disjoint_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-08-26 20:05:48,792][disjoint_time_based_config][INFO] - Quick validation succeeded.
[2025-08-26 20:05:48,817][disjoint_time_based_config][INFO] - Finalization and validation completed successfully.
[2025-08-26 20:05:48,822][cesnet_dataset][INFO] - Updating config for train set.
100%|██████████| 274/274 [00:00<00:00, 2521.14it/s]
[2025-08-26 20:05:48,934][cesnet_dataset][INFO] - Updating config for val set.
100%|██████████| 109/109 [00:00<00:00, 2865.30it/s]
[2025-08-26 20:05:48,977][cesnet_dataset][INFO] - Updating config for test set.
100%|██████████| 54/54 [00:00<00:00, 3243.58it/s]
[2025-08-26 20:05:48,996][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_1_HOUR
    Source: SourceType.INSTITUTION_SUBNETS

    Time series
        Train time series IDs: [263  84 261  72 485 ... 265 332 453 320 146], Length=274
        Val time series IDs: [486 490 317   4 176 ...  49 218 373 421 510], Length=109
        Test time series IDs: [102 447 305 414 291 ...   6  36  19 388 473], Length=54
    Time periods
        Train time periods: range(0, 3359)
        Val time periods: range(3359, 5374)
        Test time periods: range(5374, 6717)
    Features
        Taken features: ['n_flows', 'n_packets', 'n_bytes', 'sum_n_dest_asn', 'avg_n_dest_asn', 'std_n_dest_asn', 'sum_n_dest_ports', 'avg_n_dest_ports', 'std_n_dest_ports', 'sum_n_dest_ip', 'avg_n_dest_ip', 'std_n_dest_ip', 'tcp_udp_ratio_packets', 'tcp_udp_ratio_bytes', 'dir_ratio_packets', 'dir_ratio_bytes', 'avg_duration', 'avg_ttl']
        Default values: [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 

In [31]:
numpy_array, times = disjoint_dataset.get_train_numpy(workers="config")

display(numpy_array.shape)
display(times)

(274, 3359, 19)

array([datetime.datetime(2023, 10, 9, 0, 0, tzinfo=datetime.timezone.utc),
       datetime.datetime(2023, 10, 9, 1, 0, tzinfo=datetime.timezone.utc),
       datetime.datetime(2023, 10, 9, 2, 0, tzinfo=datetime.timezone.utc),
       ...,
       datetime.datetime(2024, 2, 25, 20, 0, tzinfo=datetime.timezone.utc),
       datetime.datetime(2024, 2, 25, 21, 0, tzinfo=datetime.timezone.utc),
       datetime.datetime(2024, 2, 25, 22, 0, tzinfo=datetime.timezone.utc)],
      shape=(3359,), dtype=object)